In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from IPython.display import display
import plotly.express as px

class DSP:
    def __init__(self, df):
        self.dataset = df
        self.encoders_map = {}
        

    def analise_exploratoria(self):
        display(self.dataset)
        display(self.dataset.info())

    def plot(self, df, tipo, x, y):
        if tipo == "barra":
            fig = px.histogram(
                df,
                x=x,
                color=y,
                barmode="group",
                text_auto=True,
            )
            fig.show()
        elif tipo == "pizza":
            fig = px.pie(
                df,
                names=x,
                values=y,
            )
            fig.show()
        elif tipo == "scatter":
            fig = px.scatter(
                df,
                x=x,
                y=y,
            )
            fig.show()
        elif tipo == "linha":
            fig = px.line(
                df,
                x=x,
                y=y,
            )
            fig.show()

    

    def init_encoders(self,nome_tabela):
        codificador = LabelEncoder()
        self.dataset[nome_tabela] = codificador.fit_transform(self.dataset[nome_tabela])
        self.encoders_map[nome_tabela] = codificador
        return codificador
    
    def aplicar_encoders(self, novos_dados):
        for col, enc in self.encoders_map.items():
            novos_dados[col] = enc.transform(novos_dados[col])
        return novos_dados

    def train_AI_model(self, metodo, x_treino, y_treino):
        modelos = {
            "random_forest": __import__("sklearn.ensemble").ensemble.RandomForestClassifier(),
            "arvore_decisao": __import__("sklearn.tree").tree.DecisionTreeClassifier(),
            "regressao_logistica": __import__("sklearn.linear_model").linear_model.LogisticRegression(),
            "knn": __import__("sklearn.neighbors").neighbors.KNeighborsClassifier(),
            "svm": __import__("sklearn.svm").svm.SVC(),
            "naive_bayes": __import__("sklearn.naive_bayes").naive_bayes.GaussianNB()
        }
        modelo = modelos[metodo]
        modelo.fit(x_treino, y_treino)
        return modelo
        
    def save_for_production(self,model,name_model):
        import joblib
        joblib.dump(model, f"{name_model}.pkl")
        modelo = joblib.load(f"{name_model}.pkl")
        return modelo




def run_classificacao():

    tabela = pd.read_csv(r"C:\Users\pedrovictor.veras\OneDrive - Operador Nacional do Sistema Eletrico\Documentos\ESTAGIO_ONS_PVRV_2025\GitHub\dashboard-website-template\machine_learning\classificacao\clientes.csv")

    # Analise inicial
    analise = DSP(tabela)
    #analise.analise_exploratoria()
    analise.plot(tabela,"barra","profissao","score_credito")

    # enconders
    codificador_profissao = analise.init_encoders("profissao")
    codificador_credito = analise.init_encoders("mix_credito")
    codificador_pagamento= analise.init_encoders("comportamento_pagamento")


    # y -> é a coluna da base de dados que eu quero prever
    y = tabela["score_credito"]

    # x -> as colunas da base de dados que eu vou usar pra fazer a previsão
    x = tabela.drop(columns=["score_credito", "id_cliente"])

    print("Variável númerica Alvo")
    display(y.value_counts())

    print("Variáveis categoricas (restante da tabela)")
    display(x.head(5))

    # separar em dados de treino e dados de teste
    from sklearn.model_selection import train_test_split

    x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.3)


    # treinamento do modelo
    print("treinamento do modelo...")
    modelo_arvore_decisao = analise.train_AI_model("arvore_decisao", x_treino, y_treino)
    modelo_knn = analise.train_AI_model("knn", x_treino, y_treino)

    # Passo 4 - Escolher qual o melhor modelo
    print("Prevendo...")
    previsao_arvoredecisao = modelo_arvore_decisao.predict(x_teste)
    previsao_knn = modelo_knn.predict(x_teste)

    # acurácia
    from sklearn.metrics import accuracy_score
    print("\nScore de Arvore de Decisão:")
    display(accuracy_score(y_teste, previsao_arvoredecisao))
    print("\nScore de KNN:")
    display(accuracy_score(y_teste, previsao_knn))

    # Passo 5 - Usar o melhor modelo para fazer previsão de novos clientes
    # melhor modelo é o modelo_arvoredecisao

    # importar os novos clientes para fazer a previsao
    tabela_novos_clientes = pd.read_csv(r"C:\Users\pedrovictor.veras\OneDrive - Operador Nacional do Sistema Eletrico\Documentos\ESTAGIO_ONS_PVRV_2025\GitHub\dashboard-website-template\machine_learning\classificacao\novos_clientes.csv")

    # profissao
    tabela_novos_clientes["profissao"] = codificador_profissao.transform(
        tabela_novos_clientes["profissao"])


    # mix_credito
    tabela_novos_clientes["mix_credito"] = codificador_credito.transform(
        tabela_novos_clientes["mix_credito"])

    # comportamento_pagamento
    tabela_novos_clientes["comportamento_pagamento"] = codificador_pagamento.transform(
        tabela_novos_clientes["comportamento_pagamento"])

    print("Tabela de novos clientes")
    display(tabela_novos_clientes)


    print("Previsão dos novos clientes")
    nova_previsao = modelo_arvore_decisao.predict(tabela_novos_clientes)
    display(nova_previsao)

    from sklearn.metrics import classification_report, confusion_matrix
    display(classification_report(y_teste, previsao_arvoredecisao))
    display(confusion_matrix(y_teste, previsao_arvoredecisao))



run_classificacao()

Variável númerica Alvo


score_credito
Standard    53174
Poor        28998
Good        17828
Name: count, dtype: int64

Variáveis categoricas (restante da tabela)


,mes,idade,profissao,salario_anual,num_contas,num_cartoes,juros_emprestimo,num_emprestimos,dias_atraso,num_pagamentos_atrasados,...,taxa_uso_credito,idade_historico_credito,investimento_mensal,comportamento_pagamento,saldo_final_mes,emprestimo_carro,emprestimo_casa,emprestimo_pessoal,emprestimo_credito,emprestimo_estudantil
0,1,23.0,2,19114.12,3.0,4.0,3.0,4.0,3.0,7.0,...,26.822620,265.0,21.46538,1,312.494089,1,1,1,1,0
1,2,23.0,2,19114.12,3.0,4.0,3.0,4.0,3.0,4.0,...,31.944960,266.0,21.46538,3,284.629162,1,1,1,1,0
2,3,23.0,2,19114.12,3.0,4.0,3.0,4.0,3.0,7.0,...,28.609352,267.0,21.46538,5,331.209863,1,1,1,1,0
3,4,23.0,2,19114.12,3.0,4.0,3.0,4.0,5.0,4.0,...,31.377862,268.0,21.46538,4,223.451310,1,1,1,1,0
4,5,23.0,2,19114.12,3.0,4.0,3.0,4.0,6.0,4.0,...,24.797347,269.0,21.46538,2,341.489231,1,1,1,1,0


treinamento do modelo...
Prevendo...

Score de Arvore de Decisão:


...

,mes,idade,profissao,salario_anual,num_contas,num_cartoes,juros_emprestimo,num_emprestimos,dias_atraso,num_pagamentos_atrasados,...,taxa_uso_credito,idade_historico_credito,investimento_mensal,comportamento_pagamento,saldo_final_mes,emprestimo_carro,emprestimo_casa,emprestimo_pessoal,emprestimo_credito,emprestimo_estudantil
0,1,31.0,5,19300.340,6.0,7.0,17.0,5.0,52.0,19.0,...,29.934186,218.0,44.50951,4,312.487689,1,1,0,0,0
1,4,32.0,0,12600.445,5.0,5.0,10.0,3.0,25.0,18.0,...,28.819407,12.0,0.00000,5,300.994163,0,0,0,0,1
2,2,48.0,5,20787.690,8.0,6.0,14.0,7.0,24.0,14.0,...,34.235853,215.0,0.00000,3,345.081577,0,1,0,1,0


Previsão dos novos clientes


array(['Poor', 'Poor', 'Standard'], dtype=object)

'              precision    recall  f1-score   support\n\n        Good       0.66      0.66      0.66      5326\n        Poor       0.72      0.72      0.72      8855\n    Standard       0.76      0.76      0.76     15819\n\n    accuracy                           0.73     30000\n   macro avg       0.71      0.71      0.71     30000\nweighted avg       0.73      0.73      0.73     30000\n'

array([[ 3503,   225,  1598],
       [  210,  6357,  2288],
       [ 1604,  2190, 12025]])